In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

In [2]:
design = designs.DesignFlipChip({}, overwrite_enabled=True)
gui = MetalGUI(design)

In [3]:
design.delete_all_components()

In [30]:
design.get_chip_z('Q_chip')

'5um'

In [3]:
design.chips['C_chip'].size['size_x']='8mm'
design.chips['C_chip'].size['size_y']='8mm'
design.chips['Q_chip'].size['size_x']='8mm'
design.chips['Q_chip'].size['size_y']='8mm'
design.chips['Q_chip'].size['center_z']='20um'

# Resonator length

In [4]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    #frequency in GHz
    #line_width/line_gap in um
    #N -> 2 for lambda/2, 4 for lambda/4
    
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [5]:
resonator_length_2=[]
for i in range(3):
    a=find_resonator_length(7.2+i*0.2,10,6,2)
    a=a.strip('mm')
    a=float(a)-0.8-0.2 
    resonator_length_2.append(a)
print(resonator_length_2)


[7.453254683870284, 7.224788339841421, 7.00834654020913]


# Flipchip

In [6]:
cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))

# q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
#                                             pos_y='-2000um',
#                                             chip='Q_chip',
#                                             connection_pads=cp0))]
q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
                                            pos_y='-2000um',
                                            chip='Q_chip'))]

qubits=q0

q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))


c0 = LeftCoupler(design, "c0", options=dict(pos_x=f"{q0x-60}um",
                                              pos_y=f"{q0y+80}um",
                                              pad_width="100um",
                                              pad_height="400um",
                                              cpw_stub_height="200um",
                                              orientation='270',                                            
                                              chip="C_chip"))

design.rebuild()
gui.rebuild()
gui.autoscale()

In [7]:
lpL=LaunchpadWirebondCoupled(design, 'lpL',
                                         options = dict(chip="C_chip",
                                         pos_x = '-3000um',
                                         pos_y = '2000um',
                                         lead_length = '30um'))
lpR=LaunchpadWirebondCoupled(design, 'lpR',
                                         options = dict(chip="C_chip",
                                         pos_x = '3000um',
                                         pos_y = '2000um',
                                         orientation='180',
                                         lead_length = '30um'))

In [8]:
clt0 = CoupledLineTee(design, "clt0", 
                        options=dict(pos_x='-2000um',
                                    pos_y='2000um', 
                                    coupling_length='300um',
                                    chip="C_chip",
                                    down_length='300um',
                                    fillet='99um',
                                    coupling_space='5um',
                                    mirror=True,
                                    open_termination=True))



In [9]:
fl0=RouteStraight(design, 'fl0',options=Dict(chip="C_chip", pin_inputs=Dict(start_pin=Dict(
                                                            component='lpL',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_start'))))
fl1=RouteStraight(design, 'fl1',options=Dict(chip="C_chip",pin_inputs=Dict(start_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_end'),
                                                            end_pin=Dict(
                                                            component='lpR',
                                                            pin='tie'))))


# Readout resonator

In [10]:
from collections import OrderedDict
jogs=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            spacing: str, asymmetry='0 um',  start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="C_chip",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension),
        total_length=length,
        fillet = '99.9um',
        meander=dict(asymmetry=asymmetry,
                     spacing=spacing))
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

In [11]:
rr0=connect('rr0', 'clt0', 'second_end', 'c0', 'in',
            resonator_length_2[1],jogs,jogs, 
            '400um','0um', '100um', '100um')

# EPR Analysis

In [24]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

In [25]:
hfss.start()

INFO 11:13PM [connect_project]: Connecting to Ansys Desktop API...
INFO 11:13PM [load_ansys_project]: 	Opened Ansys App
INFO 11:13PM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 11:13PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   Project23
INFO 11:13PM [connect_design]: No active design found (or error getting active design).
INFO 11:13PM [connect]: 	 Connected to project "Project23". No design detected


True

In [26]:
hfss.activate_ansys_design("flip_V5", 'eigenmode')
hfss.render_design(['Q0','c0','lpL','lpR','clt0','rr0','fl0','fl1'],
                    [('clt0','prime_end')],
                    box_plus_buffer=False)

11:14PM 15s WARNING [activate_ansys_design]: The design_name=flip_V4 was not in active project.  Designs in active project are: 
[].  A new design will be added to the project.  
INFO 11:14PM [connect_design]: 	Opened active design
	Design:    flip_V4 [Solution type: Eigenmode]
WARNING 11:14PM [connect_setup]: 	No design setup detected.
WARNING 11:14PM [connect_setup]: 	Creating eigenmode default setup.
INFO 11:14PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [27]:
hfss.close()

True

# LOM Analysis

In [12]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")


In [13]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='5.81',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '5.81',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [15]:
#c1.sim.run(components=['Q0','c0'], open_terminations=[('c0', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 09:27AM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpgbfqlw9s.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5810000000, Maxwell, 1, False
INFO 09:27AM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp8040w85r.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5810000000, Maxwell, 1, False
INFO 09:27AM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp0uj8i_yk.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5810000000, Maxwell, 2, False
INFO 09:27AM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpvosqyu2l.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5810000000, Maxwell, 3, False
INFO 09:27AM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpj9cbht_r.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5810000000, Maxwell, 4, False
INFO 09:27

,ground,pad_bot_Q0,pad_top_Q0,quad_coupler_c0
ground,355.160,-74.7610,-58.097,-162.5200
pad_bot_Q0,-74.761,107.8500,-31.540,-0.4397
pad_top_Q0,-58.097,-31.5400,107.560,-16.8050
quad_coupler_c0,-162.520,-0.4397,-16.805,180.7100


In [16]:
c1.setup.junctions = Dict({'Lj': 11, 'Cj': 2})
c1.setup.freq_readout = 6.88
c1.setup.freq_bus = []
c1.run_lom()
c1.lumped_oscillator_all

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 5.404762 [GHz]
EC 272.568559 [MHz]
EJ 14.854145 [GHz]
alpha -310.780510 [MHz]
dispersion 4.991561 [KHz]
Lq 10.995564 [nH]
Cq 71.065526 [fF]
T1 48.676264 [us]

**Coupling Properties**

tCqbus1 -8.199808 [fF]
gbus1_in_MHz -115.579158 [MHz]
χ_bus1 -3.207783 [MHz]
1/T1bus1 3269.662247 [Hz]
T1bus1 48.676264 [us]
Bus-Bus Couplings


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,5.743605,310.159747,14.854145,-357.604946,17.631943,[1.7875634154179918],[-0.0013608569130639033],0.001361,1.787563
2,5.602453,294.165765,14.854145,-337.571259,10.621421,[-188.43293205273338],[-11.776043547191183],11.776044,188.432932
3,5.560924,289.551396,14.854145,-331.822089,9.1045,[-154.79241067580506],[-7.407597236690314],7.407597,154.792411
4,5.571255,290.695461,14.854145,-333.246234,9.46238,[-145.55128204685343],[-6.6641285279825455],6.664129,145.551282
5,5.530098,286.153046,14.854145,-327.59675,8.10809,[-130.56305721176093],[-5.006745745831334],5.006746,130.563057
6,5.478465,280.511559,14.854145,-320.598783,6.657518,[-121.05111707063696],[-3.9559235274607665],3.955924,121.051117
7,5.451641,277.6058,14.854145,-317.002311,6.000518,[-119.02203906914995],[-3.6632882004345064],3.663288,119.022039
8,5.434161,275.72151,14.854145,-314.673007,5.604557,[-116.29314349852562],[-3.4013904682212983],3.401390,116.293143
9,5.416885,273.866279,14.854145,-312.381848,5.236584,[-116.25224564122205],[-3.3075702211542874],3.307570,116.252246
10,5.404762,272.568559,14.854145,-310.78051,4.991561,[-115.57915794477788],[-3.2077826796330955],3.207783,115.579158


In [17]:
c1.sim.close()

Warning! 3 COM references still alive
Ansys will likely refuse to shut down
